<a href="https://colab.research.google.com/github/AeronPanta11/Movies-Sentiment-Analysis/blob/main/Movies_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["kaggle_config_dir"]="/path/to/your/kaggle/directory"
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
100% 25.7M/25.7M [00:00<00:00, 138MB/s] 
100% 25.7M/25.7M [00:00<00:00, 130MB/s]


In [2]:
!unzip /content/imdb-dataset-of-50k-movie-reviews.zip

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [3]:
import pandas as pd
import numpy as np
df=pd.read_csv('/content/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [5]:
#changing the sentiment positive as 1 and negative as 0
df['sentiment'].replace('positive',1,inplace=True)
df['sentiment'].replace('negative',0,inplace=True)
df.head()

<ipython-input-5-c8d52f0445ef>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['sentiment'].replace('positive',1,inplace=True)
<ipython-input-5-c8d52f0445ef>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:

#split data into training and testing data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['review'],df['sentiment'],test_size=0.2,random_state=42)

In [8]:
#tokenize text data
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer= Tokenizer(num_words=5000)
# fit_on_texts should be called on the original text data
tokenizer.fit_on_texts(df['review'])

# Use the original DataFrame df to access the 'review' column
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train,maxlen=100)
X_test = pad_sequences(X_test,maxlen=100)

In [9]:
print(X_train)

[[   7    1  272 ...  206  352 3944]
 [ 152   84   22 ...   89  103    9]
 [   0    0    0 ...    2  712   64]
 ...
 [   0    0    0 ... 1611    2  601]
 [   0    0    0 ...  244  103  125]
 [ 293    8   11 ...   70   72 2066]]


In [10]:
y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


In [11]:
#lstm-long shortterm memory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout


In [12]:
#building the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

In [13]:
#compile the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [14]:
#training a model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - accuracy: 0.7589 - loss: 0.4780 - val_accuracy: 0.8595 - val_loss: 0.3158
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.8813 - loss: 0.2837 - val_accuracy: 0.8636 - val_loss: 0.3210
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9068 - loss: 0.2340 - val_accuracy: 0.8727 - val_loss: 0.3079
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9218 - loss: 0.1978 - val_accuracy: 0.8739 - val_loss: 0.3204
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9361 - loss: 0.1607 - val_accuracy: 0.8628 - val_loss: 0.3698
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9470 - loss: 0.1394 - val_accuracy: 0.8573 - val_loss: 0.3991
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.9561 - loss: 0.1221 - val_accuracy: 0.8578 - val_loss: 0.3975
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9709 - loss: 0.0847 - val_acc

In [15]:
#model Evaluation
loss,accuracy=model.evaluate(X_test,y_test)
print("loss",loss)
print("accuracy",accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8572 - loss: 0.4921
loss 0.4905373454093933
accuracy 0.8561999797821045


In [16]:
from collections.abc import Sequence
#building the predictive system
#using same process to convert text into array of integer
def predictive_system(new_review):
  new_review=tokenizer.texts_to_sequences([new_review])
  new_review=pad_sequences(new_review,maxlen=100)
  prediction=model.predict(new_review)
  if prediction[0][0]>0.5:
    sentiment="positive"
  else:
    sentiment="negative"
  return sentiment

In [17]:
#trail
new_review="this movie is amazing"
sentiment=predictive_system(new_review)
print(sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
positive
